In [1]:
import numpy as np
import dask
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import glob as glob
import time
from datetime import date
import io
import json
import os

In [2]:
prof=11
jsonfile='MEDWEST60_y2010m01d01-y2010m09d30.json'
sourcefile=open(jsonfile,'rU',encoding='utf-8')
infos=json.load(sourcefile)
nb_profilesEN4=len(infos)
list_profiles=infos.keys()

reference =  str(list(list_profiles)[prof])
lat_prof = infos[reference]['latitude']
lon_prof = infos[reference]['longitude']

with open('prof0.txt','w') as txt_file:
    txt_file.write('Profile_'+str(reference[2:10])+' '+str(lon_prof)+' '+str(lat_prof))
    
!/gpfswork/rech/egi/rote001/git/sosie/bin/ij_from_lon_lat.x -i /gpfsstore/rech/egi/commun/MEDWEST60/MEDWEST60-I/MEDWEST60_mesh_hgr.nc4 -p prof0.txt >/dev/null

with open('ij_found.out','r') as txt_file:
    last_line = txt_file.readlines()[-1]
    i0=int(last_line.split()[1])
    j0=int(last_line.split()[2])

def check_prof_in_ocean(i0,j0):
    print('check if profile is in the ocean')
    dsN=xr.open_dataset('/gpfsstore/rech/egi/commun/MEDWEST60/MEDWEST60-I/MEDWEST60_mask.nc4')
    tmaskN=dsN.tmask
    if tmaskN[0,0,int(j0),int(i0)] == 1:
        check=0
    else:
        check=1
    return check

check=check_prof_in_ocean(i0,j0)
if check == 0:
    print('profile is in the ocean')
else:
    print('profile is on the land')
            
def check_prof_date():
    print('check if profile is in the right period')
    check=0
    date_prof = infos[reference]['date']
    file_prof = infos[reference]['file']
    prof_prof = infos[reference]['profile no']
    year=int(date_prof[0:4])
    month=int(date_prof[5:7])
    day=int(date_prof[8:10])
    date=datetime.date(int(date_prof[0:4]),int(date_prof[5:7]),int(date_prof[8:10]))
    print('profile date is '+str(date))
    date_avant=date-datetime.timedelta(days=10)
    date_apres=date+datetime.timedelta(days=10)
    def date_range(start, end):
        r = (end+datetime.timedelta(days=1)-start).days
        return [start+datetime.timedelta(days=i) for i in range(r)]
    dateList = date_range(date_avant, date_apres) # list of days between datemin and datemax
    list_files=[]
    for date in dateList:
        mm="{:02d}".format(date.month) #month on 2 digits
        dd="{:02d}".format(date.day) # day on 2 digits
        file='/gpfsstore/rech/egi/commun/MEDWEST60/extracted_eNATL60/allv/MEDWEST60-BLBT02_y'+str(date.year)+'m'+str(mm)+'d'+str(dd)+'.1h_gridT.nc'
        if not os.path.exists(file):
            check=check+1
        else:
            list_files.append(file)
    return check,dateList   

check,dateList=check_prof_date()
if check == 0:
    print('profile is in the right period')
else:
    print('profile is too soon or too late')


def check_prof_near_coast(i0,j0):
    print('check if profile is too close to continent, island or boundary')
    dsN=xr.open_dataset('/gpfsstore/rech/egi/commun/MEDWEST60/MEDWEST60-I/MEDWEST60_mask.nc4')
    tmaskN=dsN.tmask
    check=0
    for i in np.arange(15):
        for j in np.arange(15):
            if (np.array(tmaskN[0,0,int(j0)-j,int(i0)-i])+np.array(tmaskN[0,0,int(j0)-j,int(i0)+i])+np.array(tmaskN[0,0,int(j0)+j,int(i0)-i])+np.array(tmaskN[0,0,int(j0)+j,int(i0)+i])) < 4:
                check=check+1
                print('selected profile is too close to continent, the program is stopping')
    return check

check=check_prof_near_coast(i0,j0)
if check == 0:
    print('profile is not too close to continent, island or boundary')
else:
    print('profile is too close to continent, island or boundary')


/gpfswork/rech/egi/rote001/git/sosie/bin/ij_from_lon_lat.x: error while loading shared libraries: libnetcdf.so.15: cannot open shared object file: No such file or directory


<ipython-input-2-19b3ec46f4a6>:3: DeprecationWarning: 'U' mode is deprecated
  sourcefile=open(jsonfile,'rU',encoding='utf-8')


check if profile is in the ocean
profile is in the ocean
check if profile is in the right period
profile date is 2010-01-14
profile is in the right period
check if profile is too close to continent, island or boundary
profile is not too close to continent, island or boundary


In [3]:
    print('check if profile has a good depth')
    dsN=xr.open_dataset('/gpfsstore/rech/egi/commun/MEDWEST60/MEDWEST60-I/MEDWEST60_mask.nc4')
    tmaskN=dsN.tmask
    prof_mod=tmaskN[0,:,int(j0),int(i0)]
    dsH=xr.open_dataset('/gpfsstore/rech/egi/commun/MEDWEST60/MEDWEST60-I/MEDWEST60_mesh_zgr.nc4')
    depthN=dsH.gdept_1d[0]


check if profile has a good depth


In [4]:
indz=np.where(prof_mod.values<1)

In [5]:
print(depthN[np.min(indz)])

<xarray.DataArray 'gdept_1d' ()>
array(332.538341)


In [6]:
    diren4="/gpfswork/rech/egi/rote001/EN4/"
    file_prof = infos[reference]['file']
    tfileEN4=diren4+file_prof



In [7]:
    dsen4=xr.open_dataset(tfileEN4)


In [8]:
ref_prof = infos[list(list_profiles)[prof]]['profile no']

In [9]:
    tempen4=dsen4['TEMP_ADJUSTED'][ref_prof]
    salten4=dsen4['PSAL_ADJUSTED'][ref_prof]
    presen4=dsen4['PRES_ADJUSTED'][ref_prof]


In [10]:
import seawater

In [12]:
depth_prof=seawater.dpth(presen4,lat_prof)

In [13]:
print(depth_prof)

[ 14.885993  25.801699  35.72457   44.654728  55.568855  65.49027
  75.41121   85.33169   95.25166  105.171165 115.09018  125.00872
 134.92677  144.84438  154.76146  164.67809  174.59422  184.5099
 194.42508  204.33977  214.25401  224.16774  234.08104  243.9938
 253.90616  263.818    273.72934  283.64023  293.55063  303.46057
 313.37     323.279    333.18747  343.09552  353.00305  362.9101
 372.81668  382.7228   392.62848  409.46695  434.2269   458.98395
 483.73798  508.48914  534.22705  557.9823   582.72455  607.4638
 632.2      653.9656          nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
       

In [14]:
indzprof=np.min(np.where(np.isnan(depth_prof)==True))

In [16]:
depth_prof[indzprof-1]

653.9656

In [17]:
print(lat_prof,lon_prof)

38.915 1.915


In [19]:
print(dsN.nav_lat[j0,i0],dsN.nav_lon[j0,i0])

<xarray.DataArray 'nav_lat' ()>
array(38.930176, dtype=float32) <xarray.DataArray 'nav_lon' ()>
array(1.940488, dtype=float32)


In [25]:
prof_box=tmaskN[0,:,int(j0)-15:int(j0)+15,int(i0)-15:int(i0)+15]

In [34]:
d,ly,lx=prof_box.shape

In [35]:
for j in np.arange(ly):
    for i in np.arange(lx):
        print(depthN[np.min(np.where(prof_box[:,j,i].values<1))])
        

<xarray.DataArray 'gdept_1d' ()>
array(343.080199)
<xarray.DataArray 'gdept_1d' ()>
array(353.779212)
<xarray.DataArray 'gdept_1d' ()>
array(332.538341)
<xarray.DataArray 'gdept_1d' ()>
array(235.885396)
<xarray.DataArray 'gdept_1d' ()>
array(127.9441)
<xarray.DataArray 'gdept_1d' ()>
array(103.061755)
<xarray.DataArray 'gdept_1d' ()>
array(97.256765)
<xarray.DataArray 'gdept_1d' ()>
array(103.061755)
<xarray.DataArray 'gdept_1d' ()>
array(115.170504)
<xarray.DataArray 'gdept_1d' ()>
array(162.782425)
<xarray.DataArray 'gdept_1d' ()>
array(343.080199)
<xarray.DataArray 'gdept_1d' ()>
array(457.035221)
<xarray.DataArray 'gdept_1d' ()>
array(506.864817)
<xarray.DataArray 'gdept_1d' ()>
array(545.784046)
<xarray.DataArray 'gdept_1d' ()>
array(586.004156)
<xarray.DataArray 'gdept_1d' ()>
array(655.863798)
<xarray.DataArray 'gdept_1d' ()>
array(714.228684)
<xarray.DataArray 'gdept_1d' ()>
array(744.215883)
<xarray.DataArray 'gdept_1d' ()>
array(774.729148)
<xarray.DataArray 'gdept_1d' ()>
a

<xarray.DataArray 'gdept_1d' ()>
array(545.784046)
<xarray.DataArray 'gdept_1d' ()>
array(599.695876)
<xarray.DataArray 'gdept_1d' ()>
array(599.695876)
<xarray.DataArray 'gdept_1d' ()>
array(613.528616)
<xarray.DataArray 'gdept_1d' ()>
array(613.528616)
<xarray.DataArray 'gdept_1d' ()>
array(641.613501)
<xarray.DataArray 'gdept_1d' ()>
array(655.863798)
<xarray.DataArray 'gdept_1d' ()>
array(699.434819)
<xarray.DataArray 'gdept_1d' ()>
array(744.215883)
<xarray.DataArray 'gdept_1d' ()>
array(805.760516)
<xarray.DataArray 'gdept_1d' ()>
array(853.261366)
<xarray.DataArray 'gdept_1d' ()>
array(885.552573)
<xarray.DataArray 'gdept_1d' ()>
array(918.333357)
<xarray.DataArray 'gdept_1d' ()>
array(951.595429)
<xarray.DataArray 'gdept_1d' ()>
array(985.330458)
<xarray.DataArray 'gdept_1d' ()>
array(1019.53008)
<xarray.DataArray 'gdept_1d' ()>
array(1019.53008)
<xarray.DataArray 'gdept_1d' ()>
array(291.95412)
<xarray.DataArray 'gdept_1d' ()>
array(343.080199)
<xarray.DataArray 'gdept_1d' ()>

array(599.695876)
<xarray.DataArray 'gdept_1d' ()>
array(613.528616)
<xarray.DataArray 'gdept_1d' ()>
array(627.501464)
<xarray.DataArray 'gdept_1d' ()>
array(655.863798)
<xarray.DataArray 'gdept_1d' ()>
array(670.251416)
<xarray.DataArray 'gdept_1d' ()>
array(559.047358)
<xarray.DataArray 'gdept_1d' ()>
array(386.812942)
<xarray.DataArray 'gdept_1d' ()>
array(559.047358)
<xarray.DataArray 'gdept_1d' ()>
array(885.552573)
<xarray.DataArray 'gdept_1d' ()>
array(968.404346)
<xarray.DataArray 'gdept_1d' ()>
array(968.404346)
<xarray.DataArray 'gdept_1d' ()>
array(968.404346)
<xarray.DataArray 'gdept_1d' ()>
array(985.330458)
<xarray.DataArray 'gdept_1d' ()>
array(985.330458)
<xarray.DataArray 'gdept_1d' ()>
array(1002.372718)
<xarray.DataArray 'gdept_1d' ()>
array(1002.372718)
<xarray.DataArray 'gdept_1d' ()>
array(1019.53008)
<xarray.DataArray 'gdept_1d' ()>
array(1019.53008)
<xarray.DataArray 'gdept_1d' ()>
array(1019.53008)
<xarray.DataArray 'gdept_1d' ()>
array(109.033005)
<xarray.Dat

<xarray.DataArray 'gdept_1d' ()>
array(714.228684)
<xarray.DataArray 'gdept_1d' ()>
array(714.228684)
<xarray.DataArray 'gdept_1d' ()>
array(714.228684)
<xarray.DataArray 'gdept_1d' ()>
array(714.228684)
<xarray.DataArray 'gdept_1d' ()>
array(714.228684)
<xarray.DataArray 'gdept_1d' ()>
array(714.228684)
<xarray.DataArray 'gdept_1d' ()>
array(729.156032)
<xarray.DataArray 'gdept_1d' ()>
array(774.729148)
<xarray.DataArray 'gdept_1d' ()>
array(805.760516)
<xarray.DataArray 'gdept_1d' ()>
array(853.261366)
<xarray.DataArray 'gdept_1d' ()>
array(869.345256)
<xarray.DataArray 'gdept_1d' ()>
array(885.552573)
<xarray.DataArray 'gdept_1d' ()>
array(901.882285)
<xarray.DataArray 'gdept_1d' ()>
array(918.333357)
<xarray.DataArray 'gdept_1d' ()>
array(934.904752)
<xarray.DataArray 'gdept_1d' ()>
array(951.595429)
<xarray.DataArray 'gdept_1d' ()>
array(951.595429)
<xarray.DataArray 'gdept_1d' ()>
array(951.595429)


In [32]:
indz_mod[:][0]

array([ 29,  29,  29, ..., 211, 211, 211])